In [1]:
import importlib

package_name = "datasets"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

datasets is not installed. Installing it now...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
datasets has been successfully installed.


In [2]:
import importlib

package_name = "accelerate"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

accelerate is not installed. Installing it now...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
accelerate has been successfully installed.


In [3]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizer, AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/6386"

# from colab.file_utils import load_required
# load_required(force_reinstall_pytorch=True)

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/12WI-y1mku9myCGqwJZuSs-bf71Ygc7th/6386


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
import os
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/12WI-y1mku9myCGqwJZuSs-bf71Ygc7th/6386


In [7]:
# !pip install datasets

In [8]:
raw_datasets = load_from_disk("finetune_data")
# checkpoint = "bert-base-uncased"
# checkpoint = "bert-base-cased"
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8925
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2232
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 80793
    })
})

In [10]:
tokenized_datasets['train']['text'][:10]

['Talentum expects that the net sales of its core business will increase in 2008 , compared to 2007 .',
 'Airline Revenues To Plunge 55% In 2020 On Coronavirus: Industry Body IATA\r\nhttps://t.co/CNzXsVOelC',
 'The total delivery volume of paper businesses in 2010 was 1,155,000 tonnes , up from 1,132,000 tonnes in 2009 .',
 'seen to outperform peers this week: NF CEG PCYC N MWIV GW VMI TM NKD GOOG MPC ADS ATHN CEN BIIB MIDD MSI',
 'RT @estherfung: Some Landlords Say Only 25% of Retail Tenants Paid April Rent https://t.co/1JuE0xG1AH via @WSJ',
 '$DIS is a long-term hold for me. Streaming and theme parks are strong. ????',
 'GMC really like how this one continues to hold up so well, volume on the up days  ',
 '$TWTR tweets in the wrong direction with slowing growth and user engagement.',
 '$FB trending nicely, intraday.',
 'The transactions would increase earnings per share in the first quarter by some EUR0 .28 .']

In [11]:
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-11-093e3287862a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


In [12]:
!pip install transformers[torch]
!pip install accelerate -U

In [13]:
import accelerate
print(accelerate.__version__)

0.24.1


In [14]:
training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=256,
    # During the first 500 training steps, the learning rate gradually increases from 0 (or a small base rate) to the specified learning rate.
    # This gradual increase helps in stabilizing the training process and often leads to better performance, as it prevents the model from making too large updates too quickly.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # how frequently the training progress is logged
    save_strategy="epoch",  # Set save strategy to "epoch" to save both best and last models
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",  # disable wandb
    # fp16=True,  # Enable mixed precision training
)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.layer.9.attention.self.value.bias', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.3.output.dense.bias', 'encoder.layer.8.output.LayerNorm.bias', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.11.attention.self.query.weight', 'encoder.layer.9.output.dense.weight', 'encoder.layer.11.attention.self.query.bias', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.9.attention.output.dense.bias', 'encoder.layer.0.output.LayerNorm.weight', 'classifier.bias', 'encoder.layer.1.attention.self.query.bias', 'encoder.layer.9.attention.output.dense.weight', 'encoder.layer.6.intermediate.dense.bias', 'encoder.layer.4.attention.output.dense.weight', 'encoder.layer.11.attention.output.dense.weight', 'encoder.layer.6.attention.self.query.weight', 'encoder.layer.4.attention.self.query.weight', 'encoder.layer.0.attent

In [15]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.562600,0.586014,0.708781,0.718858
2,0.452200,0.403054,0.821685,0.852811
3,0.227000,0.584638,0.820341,0.860230
4,0.385000,0.463715,0.823925,0.846784
5,0.183600,0.471024,0.827509,0.856182
6,0.150200,0.728569,0.808692,0.851788
7,0.110500,0.846743,0.823029,0.853105
8,0.180800,0.840219,0.813172,0.836918
9,0.159000,0.803106,0.827061,0.853899
10,0.124400,0.905620,0.812276,0.850943


TrainOutput(global_step=27900, training_loss=0.44954913011625225, metrics={'train_runtime': 4795.3771, 'train_samples_per_second': 93.058, 'train_steps_per_second': 5.818, 'total_flos': 4.026961157830512e+16, 'train_loss': 0.44954913011625225, 'epoch': 50.0})

In [16]:
output_dir = "saved_model"
os.makedirs(output_dir, exist_ok=True)

# Save the best model and the last model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [17]:
predictions = trainer.predict(tokenized_datasets["test"])
# print(predictions.predictions.shape, predictions.label_ids.shape)

In [18]:
predictions

PredictionOutput(predictions=array([[-1.9058295 ,  0.8770531 ],
       [ 1.3748249 , -0.41414675],
       [-1.915363  ,  0.8781678 ],
       ...,
       [ 0.51413834,  0.3449836 ],
       [-1.9163961 ,  0.8786872 ],
       [ 0.8281077 ,  0.12634875]], dtype=float32), label_ids=None, metrics={'test_runtime': 214.5336, 'test_samples_per_second': 376.598, 'test_steps_per_second': 1.473})

In [19]:
probabilities = softmax(predictions.predictions, axis=1)
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.0582562  0.9417438 ]
 [0.8568012  0.14319885]
 [0.05767477 0.9423252 ]
 ...
 [0.54218817 0.45781186]
 [0.05759044 0.9424096 ]
 [0.6685776  0.33142236]]
Predicted Labels:
 [1 0 1 ... 0 1 0]


In [26]:
# true_labels = tokenized_datasets["test"]["label"]

# accuracy = accuracy_score(true_labels, predicted_labels)
# precision = precision_score(true_labels, predicted_labels)
# recall = recall_score(true_labels, predicted_labels, average='weighted')
# f1 = f1_score(true_labels, predicted_labels, average='weighted')

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [27]:
# helper_model.print_wrong_classifications(predicted_labels, true_labels, raw_datasets['test'])

NameError: ignored

In [22]:
data = {'Predicted_Labels': predicted_labels}
df = pd.DataFrame(data)
df.to_csv('predicted_labels.csv', index=False)

Use the saved model

In [23]:
# Define the path to the saved model directory
saved_model_dir = "saved_model"

# Load the saved tokenizer
tokenizer = BertTokenizer.from_pretrained(saved_model_dir)

# Load the saved model
model = BertForSequenceClassification.from_pretrained(saved_model_dir)
model

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
trainer = Trainer(model=model)
predictions = trainer.predict(tokenized_datasets["test"])
# Apply softmax to convert logits to probabilities
probabilities = softmax(predictions.predictions, axis=1)

# Get the predicted class labels
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.05825622 0.9417438 ]
 [0.8568013  0.14319871]
 [0.05767477 0.9423252 ]
 ...
 [0.5421878  0.45781222]
 [0.05759043 0.9424096 ]
 [0.6685781  0.33142194]]
Predicted Labels:
 [1 0 1 ... 0 1 0]
